In [51]:
'''
install packages
'''
!pip install backtrader
!pip install yfinance
!pip install boto3

Libraries

In [33]:
import backtrader as bt
import backtrader.analyzers as btanalyzers
import pandas as pd
import yfinance as yf
import requests
from bs4 import BeautifulSoup


import matplotlib
import matplotlib.pyplot as plt

from datetime import datetime

#1

In [52]:
stocks = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'AMZN']


In [66]:
stock = yf.Ticker(stocks[1])
stock.history(period="1y")['High'][0]
stock.history(period="1y")['Close'][0]
#stock.info

332.3800048828125

General Stock Information

In [53]:
# General Info via yahoo finance website
# Initialize an empty list to store the data
data = []
for symbol in stocks:
    stock = yf.Ticker(symbol)
    stock_info = stock.info
    url = f'https://finance.yahoo.com/quote/{symbol}?p={symbol}&.tsrc=fin-srch'
    response = requests.get(url)
    if response.status_code == 200:
        stock_info = {
            'ticker': symbol,
            'website': stock_info["website"],
            'industry': stock_info["industry"],
            'sector': stock_info["sector"],
            'longBusinessSummary': stock_info["longBusinessSummary"],
        }
        data.append({
            'Ticker': stock_info['ticker'],
            'Industry': stock_info['industry'],
            'Sector': stock_info["sector"],
            'Website': stock_info['website'],
            'Description': stock_info['longBusinessSummary'],
        })

In [60]:
info_df = pd.DataFrame(data)
print(info_df)

  Ticker                        Industry                  Sector  \
0   AAPL            Consumer Electronics              Technology   
1   MSFT       Software - Infrastructure              Technology   
2  GOOGL  Internet Content & Information  Communication Services   
3   TSLA              Auto Manufacturers       Consumer Cyclical   
4   AMZN                 Internet Retail       Consumer Cyclical   

                       Website  \
0        https://www.apple.com   
1    https://www.microsoft.com   
2          https://www.abc.xyz   
3        https://www.tesla.com   
4  https://www.aboutamazon.com   

                                         Description  
0  Apple Inc. designs, manufactures, and markets ...  
1  Microsoft Corporation develops and supports so...  
2  Alphabet Inc. offers various products and plat...  
3  Tesla, Inc. designs, develops, manufactures, l...  
4  Amazon.com, Inc. engages in the retail sale of...  


Company Officer Information

In [57]:
# companyOfficers
officer_dataframes = []

# Iterate through the list of stock symbols
for symbol in stocks:
    stock = yf.Ticker(symbol)
    stock_info = stock.info

    # Check if "companyOfficers" information is available
    if 'companyOfficers' in stock_info:
        officers_data = []
        for i in range(len(stock_info['companyOfficers'])):
            officers_data.append({
                'Name': stock_info['companyOfficers'][i]['name'],
                'Title': stock_info['companyOfficers'][i]['title']
            })
        officers_df = pd.DataFrame(officers_data)
        officer_dataframes.append(officers_df)

officer_dict = {symbol: df for symbol, df in zip(stocks, officer_dataframes)}



In [59]:
for symbol, officer_df in officer_dict.items():
    print(f"Company: {symbol}")
    print(officer_df)


Company: AAPL
                      Name                                          Title
0      Mr. Timothy D. Cook                                 CEO & Director
1        Mr. Luca  Maestri                                   CFO & Sr. VP
2  Mr. Jeffrey E. Williams                        Chief Operating Officer
3   Ms. Katherine L. Adams                    Sr. VP, Gen. Counsel & Sec.
4     Ms. Deirdre  O'Brien                               Sr. VP of Retail
5         Mr. Chris  Kondo               Sr. Director of Corp. Accounting
6        Mr. James  Wilson                       Chief Technology Officer
7          Ms. Mary  Demby                      Chief Information Officer
8        Ms. Nancy  Paxton  Sr. Director of Investor Relations & Treasury
9        Mr. Greg  Joswiak                  Sr. VP of Worldwide Marketing
Company: MSFT
                          Name                                         Title
0           Mr. Satya  Nadella                                Chairman & CEO
1   

Related News List

In [39]:
# news links
news_dataframes = []

for symbol in stocks:
    stock = yf.Ticker(symbol)
    stock_info = stock.info
    news_data = stock.news
    news_data_filtered = [{'Ticker': stock_info['ticker'],'uuid': news['uuid'], 'link': news['link']} for news in news_data]

    if news_data_filtered:
        news_df = pd.DataFrame(news_data_filtered)
        news_dataframes.append(news_df)

news_dict = {symbol: df for symbol, df in zip(stocks, news_dataframes)}



In [61]:
# Access DataFrames for individual companies
for symbol, news_df in news_dict.items():
    print(f"Company: {symbol}")
    print(news_df)

Company: AAPL
                                   uuid  \
0  f8ca1409-6b0a-31d7-82b2-c5fbf1f51b10   
1  1d5cb413-7e45-3efa-bbde-7f5b1eddd3c5   
2  649cada5-a540-3629-806b-0f7c23ad1d20   
3  2e68d3ca-4548-3e13-824c-b942146c8867   
4  f96b3a40-f8e8-379e-bf2e-5e1342db508d   
5  ce9d1476-08b4-35ba-aa8f-837bd7a32cf5   
6  fa16bc22-d6b9-3360-afc4-667d543ac1b9   
7  08960913-4bd1-3168-8f1f-a40bb754ec14   

                                                link  
0  https://finance.yahoo.com/news/big-tech-profit...  
1  https://finance.yahoo.com/m/1d5cb413-7e45-3efa...  
2  https://finance.yahoo.com/news/google-antitrus...  
3  https://finance.yahoo.com/news/apple-inc-nasda...  
4  https://finance.yahoo.com/m/f96b3a40-f8e8-379e...  
5  https://finance.yahoo.com/news/little-bit-worr...  
6  https://finance.yahoo.com/m/fa16bc22-d6b9-3360...  
7  https://finance.yahoo.com/m/08960913-4bd1-3168...  
Company: MSFT
                                   uuid  \
0  a88944da-d971-3ad3-a15b-e177775da437   
1  

Converting dataframe information into different files


In [41]:
#news_df.to_csv('news_df.csv', index=False)
#news_df.to_csv('news_df.txt', sep='\t', index=False)
#news_df.to_json('news_df.json', orient='records')

Strategies

Strategy Analyzer

In [69]:
if __name__ == "__main__":

    stocks = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'AMZN']
    startdate = '2010-01-01'
    enddate = '2020-01-01'

    # Create a Backtrader Cerebro engine
    cerebro = bt.Cerebro()

    # Loop through the list of stock symbols and add data for each stock
    for symbol in stocks:
        # Fetch historical data for the stock
        data = bt.feeds.PandasData(dataname=yf.download(symbol, start=startdate, end=enddate))

        # Add the data to the cerebro engine
        cerebro.adddata(data)

    cerebro.addstrategy(SmartCross)

    cerebro.broker.setcash(1000000.0)
    
    cerebro.addsizer(bt.sizers.PercentSizer, percents = 10)
    # cerebro.addsizer(bt.sizers.SizerFix, stake=3)

    # https://www.backtrader.com/docu/analyzers/analyzers/
    cerebro.addanalyzer(btanalyzers.SharpeRatio, _name = "sharpe")
    cerebro.addanalyzer(btanalyzers.Transactions, _name = "trans")
    cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name = "trades")
    #cerebro.addanalyzer(btanalyzers.Alpha, _name = "alpha")  # attributes

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [70]:
back = cerebro.run()

cerebro.broker.getvalue()
# commisio


1184816.1444769679

In [72]:
back[0].analyzers.sharpe.get_analysis()

back[0].analyzers.trans.get_analysis()

back[0].analyzers.trades.get_analysis()
 

AutoOrderedDict([('total',
                  AutoOrderedDict([('total', 82),
                                   ('open', 1),
                                   ('closed', 81)])),
                 ('streak',
                  AutoOrderedDict([('won',
                                    AutoOrderedDict([('current', 6),
                                                     ('longest', 6)])),
                                   ('lost',
                                    AutoOrderedDict([('current', 0),
                                                     ('longest', 6)]))])),
                 ('pnl',
                  AutoOrderedDict([('gross',
                                    AutoOrderedDict([('total',
                                                      178885.0764730394),
                                                     ('average',
                                                      2208.4577342350544)])),
                                   ('net',
                            

Trading Plot

In [73]:
cerebro.plot()[0]

<IPython.core.display.Javascript object>

[<Figure size 630x560 with 13 Axes>]

In [74]:

plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False
plt.rcParams['figure.figsize']=[18, 16]
plt.rcParams['figure.dpi']=35
plt.rcParams['figure.facecolor']='w'
plt.rcParams['figure.edgecolor']='k'
cerebro.plot(iplot=False)

<IPython.core.display.Javascript object>

[[<Figure size 630x560 with 13 Axes>]]